In [ ]:
# Install required packages (no GPU dependencies)
!pip install -q transformers accelerate peft datasets pandas numpy matplotlib seaborn plotly scikit-learn faker ipywidgets

# Import libraries
import torch
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BartForConditionalGeneration,
    BartTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.metrics import f1_score, classification_report
from faker import Faker
import random
from collections import Counter
import re
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os

# Force CPU usage
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("✅ Packages installed and imported successfully!")
print(f"⚙️ Using device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.5 MB/s eta 0:00:00
✅ Packages installed and imported successfully!
⚙️ Using device: CPU


In [ ]:
class FinancialConfig:
    def __init__(self):
        # Model selection
        self.MODEL_NAME = "distilroberta-base"

        # Transaction categories
        self.CATEGORIES = [
            "Mobile Topup", "Education", "Utilities",
            "Food", "Transportation", "Healthcare",
            "Entertainment", "Shopping", "Transfer"
        ]

        # Create label mappings
        self.id2label = {i: cat for i, cat in enumerate(self.CATEGORIES)}
        self.label2id = {cat: i for i, cat in enumerate(self.CATEGORIES)}

        # Visualization colors
        self.CATEGORY_COLORS = {
            "Mobile Topup": "#FF9AA2",
            "Education": "#FFB7B2",
            "Utilities": "#FFDAC1",
            "Food": "#E2F0CB",
            "Transportation": "#B5EAD7",
            "Healthcare": "#C7CEEA",
            "Entertainment": "#F8B195",
            "Shopping": "#F67280",
            "Transfer": "#6C5B7B"
        }

        # Dataset size
        self.NUM_SAMPLES = 300

        # LoRA configuration
        self.LORA_CONFIG = LoraConfig(
            r=8,
            lora_alpha=16,
            target_modules=["query", "value"],
            lora_dropout=0.05,
            bias="none",
            task_type="SEQ_CLS"
        )

        # Training arguments
        self.TRAINING_ARGS = TrainingArguments(
            output_dir="./finetuned_model",
            learning_rate=2e-4,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            num_train_epochs=3,
            weight_decay=0.01,
            logging_dir="./logs",
            logging_steps=10,
            save_steps=100,
            eval_steps=100,
            save_total_limit=2,
            report_to="none",
            no_cuda=True
        )

# Initialize global configuration
config = FinancialConfig()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1577: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [ ]:
def generate_dataset(num_samples):
    """Generate synthetic financial transaction dataset"""
    fake = Faker()

    templates = {
        "Mobile Topup": [
            "Paid NPR {amount} for {operator} topup",
            "Recharged {operator} mobile with NPR {amount}",
            "Mobile recharge for {operator} NPR {amount}"
        ],
        "Education": [
            "Paid Rs {amount} to {institution} for tuition",
            "Sent {amount} to {institution} for course fees",
            "Education payment to {institution} Rs {amount}"
        ],
        "Utilities": [
            "Paid Rs {amount} for {utility} bill",
            "{utility} bill payment NPR {amount}",
            "Cleared {utility} dues Rs {amount}"
        ],
        "Food": [
            "Spent ₹{amount} at {restaurant}",
            "Dining at {restaurant} Rs {amount}",
            "Food order from {restaurant} ₹{amount}"
        ],
        "Transportation": [
            "Fuel at {station} Rs {amount}",
            "Taxi fare Rs {amount}",
            "Public transport pass NPR {amount}"
        ],
        "Healthcare": [
            "Medical bill at {hospital} Rs {amount}",
            "Pharmacy purchase NPR {amount}",
            "Doctor consultation fee Rs {amount}"
        ],
        "Entertainment": [
            "{service} subscription Rs {amount}",
            "Movie tickets at {cinema} NPR {amount}",
            "Concert tickets Rs {amount}"
        ],
        "Shopping": [
            "Purchase at {store} Rs {amount}",
            "Online shopping {amount}",
            "{item} bought for Rs {amount}"
        ],
        "Transfer": [
            "Sent Rs {amount} to {person}",
            "Transfer to {account} NPR {amount}",
            "Money sent to {person} ₹{amount}"
        ]
    }

    # Entity generators
    entity_generators = {
        "operator": lambda: random.choice(["NTC", "Ncell", "Smart Cell"]),
        "institution": lambda: random.choice(["Tribhuvan University", "Kathmandu University", "Whitehouse College"]),
        "utility": lambda: random.choice(["electricity", "water", "internet", "gas"]),
        "restaurant": lambda: random.choice(["Roadhouse Cafe", "OR2K", "KFC", "Pizza Hut"]),
        "station": lambda: random.choice(["Nepal Oil", "Sajha Petrol", "Shiva Fuel"]),
        "hospital": lambda: random.choice(["Norvic", "Grande", "Mediciti", "Teaching Hospital"]),
        "service": lambda: random.choice(["Netflix", "YouTube Premium", "Spotify", "Amazon Prime"]),
        "cinema": lambda: random.choice(["QFX", "Big Movies", "FCube"]),
        "store": lambda: random.choice(["Daraz", "Sastodeal", "New Road Shop"]),
        "item": lambda: random.choice(["clothes", "electronics", "groceries", "books"]),
        "person": lambda: fake.name(),
        "account": lambda: fake.bban()
    }

    data = []
    for _ in range(num_samples):
        category = random.choice(config.CATEGORIES)
        template = random.choice(templates[category])

        # Replace placeholders
        for placeholder in re.findall(r"\{(\w+)\}", template):
            if placeholder in entity_generators:
                value = entity_generators[placeholder]()
                template = template.replace(f"{{{placeholder}}}", value)

        # Add random amount
        amount = random.randint(50, 20000)
        description = template.replace("{amount}", str(amount))

        data.append({
            "description": description,
            "category": category,
            "amount": amount
        })

    return pd.DataFrame(data)

# Generate and display dataset
df = generate_dataset(config.NUM_SAMPLES)
print(f"📊 Generated {len(df)} synthetic transactions")
df.sample(5)

📊 Generated 300 synthetic transactions


,description,category,amount
53,Public transport pass NPR 6387,Transportation,6387
152,Purchase at Daraz Rs 7330,Shopping,7330
144,Movie tickets at FCube NPR 5504,Entertainment,5504
76,Money sent to Dennis Suarez ₹8987,Transfer,8987
187,Online shopping 8071,Shopping,8071


In [ ]:
# Create label mappings
label2id = {category: idx for idx, category in enumerate(config.CATEGORIES)}
id2label = {idx: category for category, idx in label2id.items()}

# Add label IDs to dataset
df['label'] = df['category'].map(label2id)

# Split dataset
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["description"],
        truncation=True,
        max_length=64,
        padding="max_length"
    )

# Tokenize datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

print(f"🚀 Training samples: {len(tokenized_train)}")
print(f"🧪 Test samples: {len(tokenized_test)}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

🚀 Training samples: 240
🧪 Test samples: 60


In [ ]:
# Load model without quantization
model = AutoModelForSequenceClassification.from_pretrained(
    config.MODEL_NAME,
    num_labels=len(config.CATEGORIES),
    id2label=id2label,
    label2id=label2id
)

# Apply LoRA
model = get_peft_model(model, config.LORA_CONFIG)
model.print_trainable_parameters()

# Metrics function
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    f1 = f1_score(labels, predictions, average="weighted")
    acc = np.mean(predictions == labels)

    report = classification_report(
        labels, predictions,
        target_names=config.CATEGORIES,
        output_dict=True
    )

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"]
    }

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 744,969 || all params: 82,870,290 || trainable%: 0.8990


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=config.TRAINING_ARGS,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

# Custom training loop
print("🚀 Starting training (CPU - might take 10-20 minutes)...")
for epoch in range(config.TRAINING_ARGS.num_train_epochs):
    # Train epoch
    train_result = trainer.train()

    # Evaluate
    eval_result = trainer.evaluate()

    # Print progress
    print(f"\n🏋️ Epoch {epoch+1}/{config.TRAINING_ARGS.num_train_epochs}")
    print(f"✅ Training Loss: {train_result.metrics['train_loss']:.4f}")
    print(f"🧪 Validation Loss: {eval_result['eval_loss']:.4f}")
    print(f"🎯 Validation Accuracy: {eval_result['eval_accuracy']:.4f}")
    print(f"🎯 Validation F1: {eval_result['eval_f1']:.4f}")

# Save model
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")
print("💾 Model saved successfully!")

/tmp/ipython-input-12-2748829808.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


🚀 Starting training (CPU - might take 10-20 minutes)...


Step,Training Loss
10,2.214100
20,2.162900
30,2.248000
40,2.170600
50,2.054100
60,1.871700
70,1.327400
80,0.767600
90,0.526700
100,0.433600



🏋️ Epoch 1/3
✅ Training Loss: 0.9371
🧪 Validation Loss: 0.0490
🎯 Validation Accuracy: 1.0000
🎯 Validation F1: 1.0000


Step,Training Loss
10,0.061200
20,0.063500
30,0.016200
40,0.017300
50,0.018800
60,0.013400
70,0.030000
80,0.088200
90,0.007200
100,0.005400



🏋️ Epoch 2/3
✅ Training Loss: 0.0250
🧪 Validation Loss: 0.0016
🎯 Validation Accuracy: 1.0000
🎯 Validation F1: 1.0000


Step,Training Loss
10,0.003500
20,0.006400
30,0.007000
40,0.014900
50,0.003200
60,0.003800
70,0.006200
80,0.001400
90,0.001500
100,0.003000



🏋️ Epoch 3/3
✅ Training Loss: 0.0036
🧪 Validation Loss: 0.0005
🎯 Validation Accuracy: 1.0000
🎯 Validation F1: 1.0000
💾 Model saved successfully!


In [ ]:
def load_model_for_inference():
    from peft import PeftModel, PeftConfig

    try:
        peft_config = PeftConfig.from_pretrained("./finetuned_model")
        base_model = AutoModelForSequenceClassification.from_pretrained(
            peft_config.base_model_name_or_path,
            num_labels=len(config.CATEGORIES),
            id2label=config.id2label,
            label2id=config.label2id
        )
        model = PeftModel.from_pretrained(base_model, "./finetuned_model")
        tokenizer = AutoTokenizer.from_pretrained("./finetuned_model")
        return model, tokenizer
    except Exception as e:
        print(f"⚠️ Error loading PEFT model: {str(e)}")
        print("🔄 Loading base model instead")

        model = AutoModelForSequenceClassification.from_pretrained(
            config.MODEL_NAME,
            num_labels=len(config.CATEGORIES),
            id2label=config.id2label,
            label2id=config.label2id
        )
        tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
        return model, tokenizer

model, tokenizer = load_model_for_inference()

# Financial text preprocessing
def preprocess_transaction(text):
    currency_map = {"npr": "₹", "rs": "₹", "rupees": "₹", "$": "USD"}
    for term, symbol in currency_map.items():
        text = text.replace(term, symbol)
    return text

# Extract amount from text
def extract_amount(text):
    matches = re.findall(r"(\d+\.?\d*)", text)
    return float(matches[0]) if matches else 0.0

# Categorize transaction
def categorize_transaction(text):
    inputs = tokenizer(
        preprocess_transaction(text),
        return_tensors="pt",
        max_length=64,
        padding=True,
        truncation=True
    )

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class_idx = logits.argmax().item()
    return model.config.id2label[predicted_class_idx]

# Initialize results DataFrame
results_df = pd.DataFrame(columns=["Transaction", "Category", "Amount"])

# UI Components
transaction_input = widgets.Textarea(
    value="Paid NPR 350 for NTC mobile topup",
    placeholder="Enter transaction description...",
    layout=widgets.Layout(width="80%", height="60px")
)

analyze_button = widgets.Button(
    description="Analyze Transaction",
    button_style="success",
    layout=widgets.Layout(width="200px")
)

clear_button = widgets.Button(
    description="Clear Results",
    button_style="warning",
    layout=widgets.Layout(width="200px")
)

output_area = widgets.Output()
results_display = widgets.Output()
charts_display = widgets.Output()

# Event handlers
def on_analyze_click(b):
    global results_df
    with output_area:
        clear_output()
        text = transaction_input.value
        if not text:
            print("⚠️ Please enter a transaction description")
            return

        category = categorize_transaction(text)
        amount = extract_amount(text)

        new_row = pd.DataFrame({
            "Transaction": [text],
            "Category": [category],
            "Amount": [amount]
        })

        results_df = pd.concat([results_df, new_row], ignore_index=True)

        print(f"✅ Transaction categorized: {text}")
        print(f"🏷️ Category: {category}")
        print(f"💰 Amount: ₹{amount:.2f}")

        update_display()

def on_clear_click(b):
    global results_df
    results_df = pd.DataFrame(columns=["Transaction", "Category", "Amount"])
    with output_area:
        clear_output()
        print("🧹 Results cleared")
    update_display()

def update_display():
    with results_display:
        clear_output()
        if not results_df.empty:
            display(HTML("<h3>Transaction History</h3>"))
            display(results_df)
        else:
            display(HTML("<p>No transactions analyzed yet</p>"))

    with charts_display:
        clear_output()
        if not results_df.empty:
            # Category distribution
            fig1 = px.pie(
                results_df,
                names='Category',
                title='Transaction Distribution by Category',
                color='Category',
                color_discrete_map=config.CATEGORY_COLORS
            )
            fig1.update_traces(textposition='inside', textinfo='percent+label')

            # Amount distribution
            fig2 = px.bar(
                results_df.groupby('Category')['Amount'].sum().reset_index(),
                x='Category',
                y='Amount',
                title='Total Amount by Category',
                color='Category',
                color_discrete_map=config.CATEGORY_COLORS
            )

            display(fig1)
            display(fig2)

# Bind events
analyze_button.on_click(on_analyze_click)
clear_button.on_click(on_clear_click)

# Display UI
display(HTML("<h2>📝 Transaction Analyzer</h2>"))
display(widgets.VBox([
    widgets.HBox([transaction_input]),
    widgets.HBox([analyze_button, clear_button]),
    output_area,
    widgets.HBox([results_display]),
    widgets.HBox([charts_display])
]))

# Initial update
update_display()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load summarization model
sum_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
sum_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Generate summary
def generate_summary(transactions):
    if transactions.empty:
        return "No transactions to summarize"

    category_counts = Counter(transactions['Category'])
    top_categories = [f"{cat} ({count})" for cat, count in category_counts.most_common(3)]

    input_text = "Financial transactions: " + "; ".join([
        f"{row['Transaction']} → {row['Category']}"
        for _, row in transactions.iterrows()
    ])

    inputs = sum_tokenizer(
        [input_text],
        max_length=1024,
        return_tensors="pt",
        truncation=True
    )

    summary_ids = sum_model.generate(
        inputs.input_ids,
        max_length=150,
        min_length=50,
        num_beams=4,
        early_stopping=True
    )

    return sum_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Summary UI
summary_button = widgets.Button(
    description="Generate Summary",
    button_style="info",
    layout=widgets.Layout(width="200px")
)

summary_output = widgets.Output()

def on_summary_click(b):
    with summary_output:
        clear_output()
        if results_df.empty:
            print("⚠️ No transactions to summarize")
            return

        summary = generate_summary(results_df)
        display(HTML("<h3>Financial Summary</h3>"))
        display(HTML(f"<div style='background-color: #e8f4f8; padding: 15px; border-radius: 5px;'>{summary}</div>"))

summary_button.on_click(on_summary_click)

display(HTML("<h2>📊 Financial Summary</h2>"))
display(widgets.HBox([summary_button]))
display(summary_output)

Output()

In [ ]:
# Budget UI
budget_header = widgets.HTML("<h2>💰 Budget Monitoring</h2>")
budget_table = widgets.Output()
alert_output = widgets.Output()

# Budget settings
budgets = {
    "Food": 5000,
    "Shopping": 4000,
    "Entertainment": 3000,
    "Transportation": 6000
}

def update_budget_display():
    with budget_table:
        clear_output()
        if results_df.empty:
            display(HTML("<p>No transactions to analyze</p>"))
            return

        # Calculate spending
        spending = results_df.groupby("Category")["Amount"].sum()

        # Create budget report
        budget_data = []
        for category, budget_limit in budgets.items():
            spent = spending.get(category, 0)
            remaining = max(0, budget_limit - spent)
            percentage = min(100, (spent / budget_limit) * 100) if budget_limit > 0 else 0

            budget_data.append({
                "Category": category,
                "Budget": f"₹{budget_limit:,.2f}",
                "Spent": f"₹{spent:,.2f}",
                "Remaining": f"₹{remaining:,.2f}",
                "Percentage": percentage
            })

        budget_df = pd.DataFrame(budget_data)

        # Display table
        display(HTML("<h4>Budget Status</h4>"))
        display(budget_df[["Category", "Budget", "Spent", "Remaining"]])

        # Progress bars
        display(HTML("<h4>Budget Utilization</h4>"))
        for _, row in budget_df.iterrows():
            color = "success" if row["Percentage"] < 75 else "warning" if row["Percentage"] < 90 else "danger"
            display(HTML(
                f"<div><strong>{row['Category']}</strong>"
                f"<div class='progress' style='height: 20px; margin-bottom: 15px;'>"
                f"<div class='progress-bar bg-{color}' role='progressbar' "
                f"style='width: {row['Percentage']}%' aria-valuenow='{row['Percentage']}' "
                f"aria-valuemin='0' aria-valuemax='100'>"
                f"{row['Percentage']:.1f}%</div></div></div>"
            ))

    with alert_output:
        clear_output()
        alerts = []
        for category, budget_limit in budgets.items():
            spent = spending.get(category, 0)
            if spent > budget_limit * 0.9:
                status = "⚠️ WARNING" if spent < budget_limit else "🚨 EXCEEDED"
                alerts.append(
                    f"{status}: {category} budget - "
                    f"Spent ₹{spent:,.2f} of ₹{budget_limit:,.2f} limit"
                )

        if alerts:
            display(HTML("<h4>Budget Alerts</h4>"))
            for alert in alerts:
                display(HTML(f"<div class='alert alert-warning'>{alert}</div>"))
        else:
            display(HTML("<p>No budget alerts at this time</p>"))

# Display budget section
display(budget_header)
display(budget_table)
display(alert_output)

# Add button to update budget display
update_budget_button = widgets.Button(
    description="Update Budget Report",
    button_style="primary",
    layout=widgets.Layout(width="200px")
)

def on_update_budget(b):
    update_budget_display()

update_budget_button.on_click(on_update_budget)
display(update_budget_button)

# Initial update
update_budget_display()

HTML(value='<h2>💰 Budget Monitoring</h2>')

Output()

Output()

Button(button_style='primary', description='Update Budget Report', layout=Layout(width='200px'), style=ButtonS…